In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta, date
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time 
import pandas as pd
from io import StringIO

In [3]:
url = "https://www.tankauction.com/"
payload = 
header= {'client_id':'yjj7479', 'password':'xodzmdhrtus0911!'}
r = requests.post(url,data = payload)
print(r.url, r.status_code)
print(r.text)

SyntaxError: invalid syntax (2714571026.py, line 2)

In [40]:
def move_to_next_page(driver):
    try:
        paging = driver.find_element(By.ID, "paging")
        # 현재 페이지 번호가 <strong> 태그로 강조되어 있음
        current_page = int(paging.find_element(By.TAG_NAME, "strong").text)
        pages = paging.find_elements(By.CSS_SELECTOR, "a.pageNo")

        next_page = None
        for page in pages:
            if int(page.text) == current_page + 1:
                next_page = page
                break

        if next_page:
            driver.execute_script("arguments[0].click();", next_page)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#lsTbody > tr.hand"))
            )
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("return document.readyState") == "complete"
            )
            time.sleep(2)
            print(f"➡️ 다음 페이지 {current_page + 1}로 이동 완료")
            return True
        else:
            print("✅ 다음 페이지가 없습니다. 크롤링 종료합니다.")
            return False

    except Exception as e:
        print(f"⚠️ 다음 페이지 이동 실패: {e}")
        return False

In [41]:


options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )
wait = WebDriverWait(driver, 10)

# --- 사이트 접속 및 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 페이지 이동 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")
result1 = []
cols = ['입찰여부','사건번호', '주소','면적','감정가','최저가','최대입찰가','입찰가','비용','실거래가','예상 낙찰비율','낙찰가','오늘조회수','누적조회수','입찰일']

while True:
    # 현재 페이지에 로딩된 tr.hand 들을 모두 가져오기
    rows = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(rows)}개")

    for idx, item in enumerate(rows, start=1):
        try:
            # tr 태그 안에서 필요한 정보 뽑기
            tid = item.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = item.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            address = driver.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
            start_price = int(driver.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
            end_price = int(driver.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
            the_date = driver.find_element(By.ID, f"bidDt_{tid}").text

            # --- 팝업 열기 ---
            main_window = driver.current_window_handle
            clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clickable)

            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            # --- 팝업 데이터 추출 ---
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
            area = driver.find_element(By.CSS_SELECTOR, '#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

            hit_info = driver.find_element(By.CSS_SELECTOR, '#hitInfo > span').text
            hit_info2 = driver.find_element(By.CSS_SELECTOR, '#hitInfo > span:nth-child(2)').text

            sale_case_rows = driver.find_elements(By.CSS_SELECTOR, "#lyCnt_add table.Ltbl_list tr")
            avg_rate = None
            for row in sale_case_rows[1:]:
                cells = row.find_elements(By.TAG_NAME, "td")
                if cells:
                    avg_rate = cells[3].text

            # --- 실거래가
            try:
                real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
                df_real = pd.read_html(StringIO(real_price_html))[0]
                real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean()*100)
            except:
                real_price = 0

            # --- 인테리어 비용
            approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")
            year = 9999
            for elem in approval_elements:
                text = elem.text.strip()
                if "사용승인일" in text:
                    year = int(text.split(":")[1].split("-")[0])
                    break

            if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000

            expense = (start_price * 0.011) + (start_price * 0.04) + 2000000 + ((start_price - end_price) * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000
            max_price = end_price + expense + 10000000
            decision = 1 if max_price < real_price else 0

            # 팝업 닫기
            driver.close()
            driver.switch_to.window(main_window)

            result1.append([
                decision, num, address, area,
                start_price, end_price, max_price,
                None, expense, real_price, avg_rate,
                None, hit_info, hit_info2, the_date
            ])

            print(f"✅ {idx}번째 tr 추출 완료")
            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")
            continue

    # --- 다음 페이지 이동 ---
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 최종 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 20개
⚠️ 1번째 tr 추출 실패: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010923e6b8 chromedriver + 6149816
1   chromedriver                        0x0000000109235cda chromedriver + 6114522
2   chromedriver                        0x0000000108cc2d5c chromedriver + 400732
3   chromedriver                        0x0000000108cc85a3 chromedriver + 423331
4   chromedriver                        0x0000000108cca63c chromedriver + 431676
5   chromedriver                        0x0000000108d530f8 chromedriver + 991480
6   chromedriver                        0x0000000108d32cb2 chromedriver + 859314
7   chromedriver                        0x0000000108d520db chromedriver + 987355
8   chromedriver

In [33]:
df

,입찰여부,사건번호,주소,면적,감정가,최저가,최대입찰가,입찰가,비용,실거래가,예상 낙찰비율,낙찰가,오늘조회수,누적조회수,입찰일
0,0,2024-54246,"서울 송파구 가락동 78-1, 15층1507호 (가락동,한화오벨리스크)",25.32㎡\n(7.659평),189000000,189000000,218074000.0,None,19074000.0,1917083,None,None,10,233,25.04.28
1,0,2023-4493(2),"서울 동대문구 이문동 327-7, 14층1412호 (이문동,세진빌딩) 외 6필지",21.73㎡\n(6.573평),329000000,263200000,316319000.0,None,43119000.0,3000000,None,None,3,276,25.04.29
2,0,2023-125267,"서울 영등포구 영등포동1가 145, 제9층 제913호",14.89㎡\n(4.504평),134000000,43909000,89623475.0,None,35714475.0,1242333,None,None,3,830,25.04.29
3,0,2024-3282,"서울 도봉구 창동 811, 101동 2층205호 (도봉동,대동아파트)",59.76㎡\n(18.077평),485000000,388000000,458435000.0,None,60435000.0,4033333,None,None,3,286,25.04.29
4,0,2024-106495,"서울 도봉구 도봉동 641, 114동 11층1105호 (도봉동,서원아파트) 외 1필지",39.85㎡\n(12.055평),399000000,319200000,380089000.0,None,50889000.0,3175833,86.59,None,3,232,25.04.29
5,0,2024-109326,"서울 동대문구 전농동 6, 16동 1층117호 (전농동,전농우성아파트) 외 1필지",44.24㎡\n(13.383평),536000000,343040000,438432000.0,None,85392000.0,4682500,None,None,3,580,25.04.29
6,0,2024-111985,"서울 도봉구 방학동 272, 16동 12층1210호 (방학동,신동아아파트)",53.16㎡\n(16.081평),374000000,299200000,357314000.0,None,48114000.0,0,86.24,None,3,346,25.04.29
7,0,2024-112223,"서울 노원구 상계동 639, 103동 12층1213호 (상계동,보람아파트) 외 1필지",44.33㎡\n(13.41평),411000000,328800000,391021000.0,None,52221000.0,3914167,89.66,None,3,361,25.04.29
8,0,2024-114462,"서울 강서구 방화동 832, 제303동 제4층 제408호 (방화동, 방화아파트)",전체 59.04㎡\n(17.86평)\n중 지분:19.68㎡\n(5.953평),189000000,151200000,188779000.0,None,27579000.0,0,None,None,3,468,25.04.29
9,0,2024-142900,"서울 강서구 화곡동 938-23, 제8층 제802호 (화곡동, 에스엠케이아파트)",28.01㎡\n(8.473평),294000000,294000000,330004000.0,None,26004000.0,0,None,None,3,64,25.04.30


In [36]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요 경기
주거용과 아파트 중 하나를 입력해주세요 아파트


In [22]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)
# 로그인
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# 경매 검색
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()

Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# 날짜 선택
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# 검색 버튼 클릭
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()

# 리스트 갯수 조정
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')

# 리스트 항목 로딩
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
# 렌더링이 완전히 끝났는지 확인 (JavaScript DOM 상태 체크)
WebDriverWait(driver, 10).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)


print("🔍 리스트 로딩 완료")

# row_id 수집
rows = driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand')
time.sleep(2)
# 각 row에서 데이터 추출
result1 = []
# 입찰가
price = []
# 낙찰가
award_price = []
# 컬럼명
cols = ['입찰여부','사건번호', '주소','면적','감정가','최저가','최대입찰가','입찰가','비용', '실거래가','예상 낙찰비율', '낙찰가', '오늘조회수','누적조회수','입찰일']
for item in rows:
    try:
        tid = item.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
        num = item.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
        address = item.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
        start_price = int(item.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
        end_price = int(item.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
        the_date = item.find_element(By.ID, f"bidDt_{tid}").text

        view_td = item.find_elements(By.CSS_SELECTOR, 'td')[-1]
        view_divs = view_td.find_elements(By.TAG_NAME, 'div')
        

        # 팝업 열기 및 닫기
        main_window = driver.current_window_handle
        clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
        driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", clickable)
        WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
        new_window = [w for w in driver.window_handles if w != main_window][0]
        driver.switch_to.window(new_window)
        # 팝업 데이터 추출 위치

        # 팝업 : 면적
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
        area = driver.find_element(By.CSS_SELECTOR,'#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

        # 팝업 : 조회수
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#hitInfo")))
        hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
        hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
        # hit_info3 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(3)').text


        # 팝업 : 매각사례
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_add")))
        example_1 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(1)').text.split('(')[1].split('건')[0]
        # 예상 낙찰비율
        price_rate_avg = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(4)').text.split('%')[0]
        example_2 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(5)').text.split('명')[0]

        # 팝업: 국토부 실거래가
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_add")))
            example_1 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(1)').text.split('(')[1].split('건')[0]
            price_rate_avg = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(4)').text.split('%')[0]
            example_2 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(5)').text.split('명')[0]
        except Exception as e:
            print(f"⚠️ 매각사례 없음: {row_id}")
            example_1 = None
            price_rate_avg = None
            example_2 = None

        # 내부 인테리어 비용 계산
        approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")

        approval_years = []

        year = 9999  # 기본값 (매우 큰 값으로 초기화)
        for elem in approval_elements:
            text = elem.text.strip()
            if "사용승인일" in text:
                year = int(text.split(":")[1].split("-")[0])
                break  # 첫 번째 것만 쓰니까 찾으면 바로 탈출

        if year < 2010:
            size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
            try:
                area_size = int(size_text.split("(")[1].split("평")[0])
                interior = area_size * 1000000
            except:
                interior = 3000000  # 혹시 파싱 실패하면 디폴트
        else:
            interior = 3000000
        # 최종 비용 계산
        expense = (start_price * 0.011) + (start_price * 0.04) + 2000000 + ((start_price - end_price) * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000

        # 최대입찰가
        max_pice = end_price + expense + 10000000

        # 입찰 여부 판단
        decision = 1 if max_pice < real_price else 0

        result1.append([
            decision, num, address, area, start_price, end_price, max_pice, 
            None, expense, real_price, price_rate_avg, None, hit_info, hit_info2, the_date])
        driver.close()
        driver.switch_to.window(main_window)
        time.sleep(2)           
        print(f"{row_id}_추출 완료")

    except Exception as e:
        print(f"⚠️ {row_id}를 찾을 수 없습니다. 스킵합니다.")
        continue

cols = ['입찰여부','사건번호', '주소','면적','감정가','최저가','최대입찰가','입찰가','비용', '실거래가','예상 낙찰비율', '낙찰가', '오늘조회수','누적조회수','입찰일']
df = pd.DataFrame(result1, columns=cols)
df
# df.to_csv(f"../Users/jihyun/Desktop/자료/경매/입찰결정/[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매.csv", encoding="utf-8-sig", index=False)



🔍 리스트 로딩 완료
Tr_20_추출 완료
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합니다.
⚠️ Tr_20를 찾을 수 없습니다. 스킵합

,입찰여부,사건번호,주소,면적,감정가,최저가,최대입찰가,입찰가,비용,실거래가,예상 낙찰비율,낙찰가,오늘조회수,누적조회수,입찰일
0,0,2024-52691,"서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지",16.685㎡\n(5.047평),120000000,120000000,144520000.0,None,14520000.0,11008,102.96,None,5,173,25.04.28


In [11]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)
# 로그인
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# 경매 검색
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()

Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# 날짜 선택
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# 검색 버튼 클릭
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()

# 리스트 갯수 조정
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')

# 리스트 항목 로딩
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
# 렌더링이 완전히 끝났는지 확인 (JavaScript DOM 상태 체크)
WebDriverWait(driver, 10).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)


print("🔍 리스트 로딩 완료")

# row_id 수집
row_ids = []
for retry in range(3):
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand')
        for row in rows:
            row_ids.append(row.get_attribute("id"))
        break
    except StaleElementReferenceException:
        print("⚠️ StaleElement 발생, row 재시도 중...")
        time.sleep(1)
# 입찰가
price = []
# 낙찰가
award_price = []

# 리스트 데이터 수집 (임시)
item = driver.find_element(By.CSS_SELECTOR, '#lsTbody > tr.hand')
tid = item.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
num = item.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
address = item.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
start_price = int(item.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
end_price = int(item.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
the_date = item.find_element(By.ID, f"bidDt_{tid}").text

view_td = driver.find_elements(By.CSS_SELECTOR, 'td')[-1]
view_divs = driver.find_elements(By.TAG_NAME, 'div')
        


# 팝업 열기 (임시)
main_window = driver.current_window_handle
clickable = driver.find_element(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')
driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
time.sleep(1)
driver.execute_script("arguments[0].click();", clickable)

# 팝업창 전환
WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
new_window = [w for w in driver.window_handles if w != main_window][0]
driver.switch_to.window(new_window)

# 팝업 : 면적
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
area = driver.find_element(By.CSS_SELECTOR,'#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

# 팝업 : 조회수
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#hitInfo")))
hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
# hit_info3 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(3)').text


# 팝업 : 매각사례
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_add")))
example_1 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(1)').text.split('(')[1].split('건')[0]
# 예상 낙찰비율
price_rate_avg = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(4)').text.split('%')[0]
example_2 = driver.find_element(By.CSS_SELECTOR,'#lyCnt_add > table:nth-child(8) > tbody > tr:nth-child(2) > td:nth-child(5)').text.split('명')[0]

# 팝업: 국토부 실거래가
real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
df_real = pd.read_html(StringIO(real_price_html))[0]
real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean())

# 내부 인테리어 비용 계산
approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")

approval_years = []

for elem in approval_elements:
    text = elem.text.strip()
    if "사용승인일" in text:
        # 사용승인일:2012-11-21 → 2012만 추출
        year = int(text.split(":")[1].split("-")[0])
        approval_years.append(year)

if year < 2010:
    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
    try:
        area_size = int(size_text.split("(")[1].split("평")[0])
        interior = area_size * 1000000
    except:
        interior = 3000000  # 혹시 파싱 실패하면 디폴트
else:
    interior = 3000000
# 최종 비용 계산
expense = (start_price * 0.011) + (start_price * 0.04) + 2000000 + ((start_price - end_price) * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000

# 최대입찰가
max_pice = end_price + expense + 10000000

# 입찰 여부 판단
decision = 1 if max_pice < real_price else 0
    
print(decision, num, address, area, start_price, end_price, max_pice, price, expense, real_price, price_rate_avg, award_price, hit_info, hit_info2, the_date)


🔍 리스트 로딩 완료
⚠️ StaleElement 발생, row 재시도 중...
0 2024-52691 서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지 16.685㎡
(5.047평) 120000000 120000000 144520000.0 [] 14520000.0 11008.333333333334 102.96 [] 15 15 25.04.28


In [15]:
today.month:02d

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1056305615.py, line 1)

In [ ]:
#qtLst > tbody > tr > td > div:nth-child(1)

#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)

In [ ]:
df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")))[0] 

In [ ]:
df['평균가']

In [20]:


# 로그인 폼 채우기
driver.find_element(By.ID, 'client_id').send_keys('yjj7479')
time.sleep(2)
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()

time.sleep(5)  # 로그인 처리 대기


In [112]:
# 팝업 열기 및 닫기
main_window = driver.current_window_handle
clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
time.sleep(1)
driver.execute_script("arguments[0].click();", clickable)

WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
new_window = [driver.window_handles][0]
driver.switch_to.window(new_window)
# 팝업 데이터 추출 위치
driver.close()
driver.switch_to.window(main_window)


InvalidArgumentException: Message: invalid argument: 'handle' must be a string
  (Session info: chrome=125.0.6422.61)
Stacktrace:
0   chromedriver                        0x000000010fc096b8 chromedriver + 6149816
1   chromedriver                        0x000000010fc00cda chromedriver + 6114522
2   chromedriver                        0x000000010f68dd5c chromedriver + 400732
3   chromedriver                        0x000000010f70e80a chromedriver + 927754
4   chromedriver                        0x000000010f6fdc78 chromedriver + 859256
5   chromedriver                        0x000000010f6cd6d5 chromedriver + 661205
6   chromedriver                        0x000000010f6cdf6e chromedriver + 663406
7   chromedriver                        0x000000010fbcacf0 chromedriver + 5893360
8   chromedriver                        0x000000010fbd04bc chromedriver + 5915836
9   chromedriver                        0x000000010fbac8b4 chromedriver + 5769396
10  chromedriver                        0x000000010fbd0

In [34]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

# 경매 검색 
btn_search = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, ".menuicon_10")
    ))
btn_search.click()
# 지역 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd"))))
siCd_dropdown.select_by_visible_text(keyword1)

# 물건종류 선택
time.sleep(1)
ctgr_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr"))))
ctgr_dropdown.select_by_visible_text(keyword2)

time.sleep(1)
# 최대가격 선택
minbAmtEnd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
minbAmtEnd_dropdown.select_by_visible_text('4억')

지역을 입력해주세요서울


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00C88073+60707]
	GetHandleVerifier [0x00C880B4+60772]
	(No symbol) [0x00AB0683]
	(No symbol) [0x00AF8660]
	(No symbol) [0x00AF89FB]
	(No symbol) [0x00B41022]
	(No symbol) [0x00B1D094]
	(No symbol) [0x00B3E824]
	(No symbol) [0x00B1CE46]
	(No symbol) [0x00AEC5D3]
	(No symbol) [0x00AED424]
	GetHandleVerifier [0x00ECBB53+2435075]
	GetHandleVerifier [0x00EC70F3+2416035]
	GetHandleVerifier [0x00EE349C+2531660]
	GetHandleVerifier [0x00C9F145+155125]
	GetHandleVerifier [0x00CA5AED+182173]
	GetHandleVerifier [0x00C8F948+91640]
	GetHandleVerifier [0x00C8FAF0+92064]
	GetHandleVerifier [0x00C7A5B0+4704]
	BaseThreadInitThunk [0x76FD5D49+25]
	RtlInitializeExceptionChain [0x77BFCF0B+107]
	RtlGetAppContainerNamedObjectPath [0x77BFCE91+561]


In [101]:
driver.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1].text

'아파트\n2024-52691\n서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지\n(서울 강동구 천중로42길 70)\n건물 16.685㎡(5.047평), 대지권 4.914㎡(1.486평)\n토지·건물 일괄매각  임차권등기,대항력 있는 임차인,공시가 1억이하'

In [15]:
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

# 물건종류 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr"))))
siCd_dropdown.select_by_visible_text(keyword2)


주거용과 아파트 중 하나를 입력해주세요 아파트


In [18]:
# 최대가격 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
siCd_dropdown.select_by_visible_text('4억')

In [52]:
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")
# 시작일자 선택
today = datetime.today()
gnDt_datepicker = wait.until(EC.element_to_be_clickable((By.ID, "bgnDt")))
gnDt_datepicker.clear()
gnDt_datepicker.send_keys(start_str)

# 끝일자 선택
endDt_datepicker = wait.until(EC.element_to_be_clickable((By.ID, "endDt")))
endDt_datepicker.clear()
endDt_datepicker.send_keys(end_str)

# 건물면적 최대선택
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# 검색버튼
time.sleep(3)
btnSrch = wait.until(EC.element_to_be_clickable((By.ID, "btnSrch")))
btnSrch.click()

# 리스트 갯수 늘리기
dataSize_s_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s"))))
dataSize_s_dropdown.select_by_visible_text('100')

In [ ]:
# 끝일자 선택
endDt_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "endDt"))))
endDt_dropdown.select_by_visible_text(end_str)

In [ ]:
# 최대가격 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
siCd_dropdown.click()
siCd_dropdown.select_by_visible_text('4억')

In [49]:
# 검색버튼
time.sleep(2)
btnSrch = wait.until(EC.element_to_be_clickable((By.ID, "btnSrch")))
btnSrch.click()

In [35]:
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

In [51]:
result1 = []
for item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
    # 사건번호
    num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
    # 주소
    addr_div = item_list.find_element(By.CSS_SELECTOR, 'div[id^="adrs_"]')
    address = addr_div.get_attribute("innerText")
    # 감정가
    start_price = int(item_list.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))
    #최저가
    end_price = int(item_list.find_element(By.ID, "minbAmt_2239070").text.replace(",",""))
    # 입찰일
    the_date = item_list.find_element(By.ID, "bidDt_2239070").text
    # 조회수
    pop = item_list.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text
    print(num,address,start_price,end_price,the_date,pop)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="apslAmt_2239070"]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100c126b8 chromedriver + 6149816
1   chromedriver                        0x0000000100c09cda chromedriver + 6114522
2   chromedriver                        0x0000000100696d5c chromedriver + 400732
3   chromedriver                        0x00000001006e3aa5 chromedriver + 715429
4   chromedriver                        0x00000001006e3bf1 chromedriver + 715761
5   chromedriver                        0x00000001006d8106 chromedriver + 667910
6   chromedriver                        0x0000000100706cdd chromedriver + 859357
7   chromedriver                        0x00000001006d7fe8 chromedriver + 667624
8   chromedrive

In [83]:

main_window = driver.current_window_handle

# 리스트버튼
time.sleep(2)
clickable = driver.find_element(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')
clickable.click()

# 새로운 창으로 전환
WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
new_window = [window for window in driver.window_handles if window != main_window][0]
driver.switch_to.window(new_window)

# 데이터 추출


# 팝업창 닫기
driver.close()

# 다시 원래 창으로 돌아가기
driver.switch_to.window(main_window)


In [87]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/ca/caView.php?tid=2273798&chkNo=1&TotNo=49')

wait = WebDriverWait(driver, 10)

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#hitInfo')

In [86]:
driver.find_element(By.CSS_SELECTOR, '#hitInfo')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#hitInfo"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010f6726b8 chromedriver + 6149816
1   chromedriver                        0x000000010f669cda chromedriver + 6114522
2   chromedriver                        0x000000010f0f6d5c chromedriver + 400732
3   chromedriver                        0x000000010f143aa5 chromedriver + 715429
4   chromedriver                        0x000000010f143bf1 chromedriver + 715761
5   chromedriver                        0x000000010f188dd4 chromedriver + 998868
6   chromedriver                        0x000000010f166cdd chromedriver + 859357
7   chromedriver                        0x000000010f1860db chromedriver + 987355
8   chromedriver             

In [ ]:
driver.switch_to.window(driver.window_handles[-1])  

In [79]:
result1 = []
for item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
    try:
        # 고유 물건 ID 추출 (예: 2239070)
        tid = item_list.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
        
        # 사건번호
        num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text

        # 주소
        addr_div = item_list.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}')
        address = addr_div.get_attribute("innerText")

        # 감정가
        start_price = int(item_list.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))

        # 최저가
        end_price = int(item_list.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))

        # 입찰일
        the_date = item_list.find_element(By.ID, f"bidDt_{tid}").text

        # 결과 저장
        result1.append([num, address, start_price, end_price, the_date, pop])


        

        # 출력
        print(num, address, start_price, end_price, the_date)

    except Exception as e:
        print("❗ 오류 발생:", e)

2024-52691 서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지 120000000 120000000 25.04.28 
2024-54246 서울 송파구 가락동 78-1, 15층1507호 (가락동,한화오벨리스크) 189000000 189000000 25.04.28 
2023-4493(2) 서울 동대문구 이문동 327-7, 14층1412호 (이문동,세진빌딩) 외 6필지 329000000 263200000 25.04.29 
2023-125267 서울 영등포구 영등포동1가 145, 제9층 제913호 134000000 43909000 25.04.29 
2024-3282 서울 도봉구 창동 811, 101동 2층205호 (도봉동,대동아파트) 485000000 388000000 25.04.29 
2024-106495 서울 도봉구 도봉동 641, 114동 11층1105호 (도봉동,서원아파트) 외 1필지 399000000 319200000 25.04.29 
2024-109326 서울 동대문구 전농동 6, 16동 1층117호 (전농동,전농우성아파트) 외 1필지 536000000 343040000 25.04.29 
2024-111985 서울 도봉구 방학동 272, 16동 12층1210호 (방학동,신동아아파트) 374000000 299200000 25.04.29 
2024-112223 서울 노원구 상계동 639, 103동 12층1213호 (상계동,보람아파트) 외 1필지 411000000 328800000 25.04.29 
2024-114462 서울 강서구 방화동 832, 제303동 제4층 제408호 (방화동, 방화아파트) 189000000 151200000 25.04.29 
2024-142900 서울 강서구 화곡동 938-23, 제8층 제802호 (화곡동, 에스엠케이아파트) 294000000 294000000 25.04.30 
2023-107195 서울 관악구 신림동 1421-1, 10층1004호 (신림동,신림큐브) 140000000 15032

In [106]:
driver.find_elements(By.CSS_SELECTOR, ".hand")[1].get_attribute("innerText")

''

In [44]:
driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand')[60].find_element(By.ID, "adrs_2239070").get_attribute("innerText")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="adrs_2239070"]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b8526b8 chromedriver + 6149816
1   chromedriver                        0x000000010b849cda chromedriver + 6114522
2   chromedriver                        0x000000010b2d6d5c chromedriver + 400732
3   chromedriver                        0x000000010b323aa5 chromedriver + 715429
4   chromedriver                        0x000000010b323bf1 chromedriver + 715761
5   chromedriver                        0x000000010b318106 chromedriver + 667910
6   chromedriver                        0x000000010b346cdd chromedriver + 859357
7   chromedriver                        0x000000010b317fe8 chromedriver + 667624
8   chromedriver  

In [42]:
driver.find_element(By.CSS_SELECTOR, ".right")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".clear.f15.bold"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b8526b8 chromedriver + 6149816
1   chromedriver                        0x000000010b849cda chromedriver + 6114522
2   chromedriver                        0x000000010b2d6d5c chromedriver + 400732
3   chromedriver                        0x000000010b323aa5 chromedriver + 715429
4   chromedriver                        0x000000010b323bf1 chromedriver + 715761
5   chromedriver                        0x000000010b318106 chromedriver + 667910
6   chromedriver                        0x000000010b346cdd chromedriver + 859357
7   chromedriver                        0x000000010b317fe8 chromedriver + 667624
8   chromedriver      

In [83]:
int(driver.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))

372000000

In [56]:
driver.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=125.0.6422.61)
Stacktrace:
0   chromedriver                        0x0000000100c126b8 chromedriver + 6149816
1   chromedriver                        0x0000000100c09cda chromedriver + 6114522
2   chromedriver                        0x0000000100696d5c chromedriver + 400732
3   chromedriver                        0x000000010067c646 chromedriver + 292422
4   chromedriver                        0x000000010067c533 chromedriver + 292147
5   chromedriver                        0x0000000100699172 chromedriver + 409970
6   chromedriver                        0x0000000100725abb chromedriver + 985787
7   chromedriver                        0x0000000100706a53 chromedriver + 858707
8   chromedriver                        0x00000001006d66d5 chromedriver + 661205
9   chromedriver                        0x00000001006d6f6e chromedr

In [62]:
driver.find_element(By.ID,"saNo_2273798").text

'2024-52691'

In [ ]:

item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
# 사건번호
num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
# 주소
addr_div = item_list.find_element(By.CSS_SELECTOR, 'div[id^="adrs_"]')
address = addr_div.get_attribute("innerText")
# 감정가
start_price = int(item_list.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))
#최저가
end_price = int(item_list.find_element(By.ID, "minbAmt_2239070").text.replace(",",""))
# 입찰일
the_date = item_list.find_element(By.ID, "bidDt_2239070").text
# 조회수
pop = item_list.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text
print(num,adress,start_price,end_price,the_date,pop)

In [63]:
driver.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text

'2024-52691'

In [74]:
driver.find_element(By.ID, "apslAmt_2273798").text

'120,000,000'

# 나중에 셀렉트 실행해볼것

In [ ]:
## Ex 5-5. QComboBox.

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QComboBox


class MyApp(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.lbl = QLabel('Option1', self)
        self.lbl.move(50, 150)

        cb = QComboBox(self)
        cb.addItem('Option1')
        cb.addItem('Option2')
        cb.addItem('Option3')
        cb.addItem('Option4')
        cb.move(50, 50)

        cb.activated[str].connect(self.onActivated)

        self.setWindowTitle('QComboBox')
        self.setGeometry(300, 300, 300, 200)
        self.show()

    def onActivated(self, text):
        self.lbl.setText(text)
        self.lbl.adjustSize()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())